![Vacation image](https://ackcent.com/img/2018/07/cybersecurity-holiday-tips.jpeg)
image credit: https://ackcent.com 

# VacationPy

In [1]:
# Enabling widgets extensions to Jupyter: (uncomment if needed)
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# Installing gmaps (uncomment if needed)
#!pip install gmaps
# Telling Jupyter to load the extension with: (uncomment if needed)
#!jupyter nbextension enable --py gmaps
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import g_key
print("All necessary libraries installed")

All necessary libraries installed


## Store the cities obtained from WeatherPy into DataFrame
### Loading Cities.csv file from Output_Data folder to a DataFrame

In [2]:
# Location of the input data file
data_file = "Output_Data/Cities.csv"
# Use Pandas to read data
df_cities = pd.read_csv(data_file)
# Display the first 5 rows of the dataframe
print('\033[96m'+'\033[1m'+"The shape of the {} dataframe is: {}" .format("df_cities",df_cities.shape)+'\033[0m')
df_cities.head()

The shape of the df_cities dataframe is: (583, 10)


,City,Country,Date,Latitude,Longitude,Humidity,Pressure,Max_Temp,Cloudiness,Wind_Speed
0,Ushuaia,AR,2020-06-16 17:35:18,-54.80,-68.30,81,1004,37.40,40,2.1
1,Hare Bay,CA,2020-06-16 17:35:18,48.85,-54.01,35,1021,71.60,75,6.2
2,Wolfsburg,DE,2020-06-16 17:35:07,52.43,10.80,73,1013,71.01,58,1.5
3,Klaksvík,FO,2020-06-16 17:35:19,62.23,-6.59,93,1017,51.80,100,5.1
4,Saint George,US,2020-06-16 17:34:00,37.10,-113.58,18,1011,90.00,1,12.3


# Humidity Heatmap
* Configuring gmaps.
* Using Latitude and Longitude as locations and Humidity as the weight.
* Adding Heatmap layer to map.

### Selecting data and weight values for plotting

In [3]:
# Store latitude and longitude in locations
locations = df_cities[["Latitude", "Longitude"]]
# Filling NaN values and converting "humidity" to float
humidity = df_cities["Humidity"].astype(float) # Humidity will be used as weight

### Configuring gmap and adding Humidity Heatmap layer

In [4]:
import gmaps
# Let's configure gmap using api_key
gmaps.configure(api_key=g_key)
# Configuring the layout of the map
figure_layout = {
    "width":"1200px",
    "height":"600px",
    "border":"1px solid black",
    "padding":"1px"
}
# Starting co-ordinates (center of world map)
center_coordinates = (30.0333,31.2333)
fig = gmaps.figure(center=center_coordinates,map_type="ROADMAP",zoom_level=2.2,layout=figure_layout)
# Create heat layer based on humidity
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
# Add layer
fig.add_layer(heatmap_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1200px'))

## Narrowing the DataFrame to find our ideal weather conditions.  
### Ideal conditions are : 
- A max temperature lower than 80 degrees but higher than 70.
- Wind speed less than 10 mph.
- Zero cloudiness.
- Drop any rows with null values and those which don't contain all three conditions. 

In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None # Prevents warnings during "chained assignments"
# Let's make a copy of the dataframe
df_weather = df_cities.copy()
# Filling NaN values and converting "Cloudiness" to int
df_weather["Cloudiness"] = df_weather["Cloudiness"].astype(float) 
df_weather["Wind_Speed"] = df_weather["Wind_Speed"].astype(float) 
# Let's get the indexes where weather is ideal
SelectList = df_weather[(df_weather.Max_Temp > 70) & (df_weather.Max_Temp < 80)\
                       & (df_weather.Cloudiness == 0) & (df_weather.Wind_Speed < 10)].index
# Let's create a new dataframe after filtering
df_mycities = df_weather[df_weather.index.isin(SelectList)]
# Let's drop rows with missing values
df_mycities.dropna(inplace=True)
# Let's reset index in the new dataframe
df_mycities.reset_index(drop = True,inplace=True)
print('\033[96m'+'\033[1m'+"The shape of the {} dataframe with ideal weather conditions {}:" \
      .format("df_mycities",df_mycities.shape)+'\033[0m')
df_mycities.head(15) # Displaying the dataframe

The shape of the df_mycities dataframe with ideal weather conditions (13, 10):


,City,Country,Date,Latitude,Longitude,Humidity,Pressure,Max_Temp,Cloudiness,Wind_Speed
0,Kununurra,AU,2020-06-16 17:35:26,-15.77,128.73,28,1016,77.00,0.0,4.60
1,Benguela,AO,2020-06-16 17:35:44,-12.58,13.41,81,1014,71.55,0.0,1.15
2,Weleri,ID,2020-06-16 17:35:49,-6.97,110.07,79,1011,77.14,0.0,1.51
3,Ji’an,CN,2020-06-16 17:35:56,27.12,114.98,75,1004,79.79,0.0,4.09
4,Morondava,MG,2020-06-16 17:35:57,-20.28,44.28,71,1018,75.72,0.0,6.27
5,Sirte,LY,2020-06-16 17:26:23,31.21,16.59,72,1016,73.06,0.0,6.39
6,Kamenka,RU,2020-06-16 17:26:33,51.32,42.77,44,1009,75.00,0.0,0.45
7,Labuhan,ID,2020-06-16 17:36:34,-6.88,112.21,82,1011,77.76,0.0,3.18
8,Erzin,TR,2020-06-16 17:36:37,36.96,36.20,82,1009,75.00,0.0,2.24
9,Detchino,RU,2020-06-16 17:36:38,54.81,36.31,57,1017,75.20,0.0,3.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Let's make a copy of the dataframe and keep only the relevant columns
hotel_df = df_mycities[["City","Country","Latitude","Longitude"]]
# Creating additional column for hotel name information
hotel_df.insert(loc = 4,column = "Name",value=["" for i in range(hotel_df.shape[0])])

## Studying the response of the google place API using Austin co-ordinates

In [7]:
import json
# Find the closest hotel for co-ordinates in Austin
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
austin_coords = "30.26666,-97.73333"
params = {
    "location": austin_coords,  
    "radius":5000,
    "type": "hotel",
    "key": g_key,
    }
# make request and print url
austin_hotel = requests.get(base_url, params=params).json()
# Uncomment to pretty print the json 
# print(json.dumps(austin_hotel, indent=4, sort_keys=True))

In [8]:
# Let's check number of results obtained by this call
print('\033[94m'+'\033[1m'+"No of restaurants within 5km of {}: {}".format("Austin",len(austin_hotel["results"]))+'\033[0m')

No of restaurants within 5km of Austin: 20


In [9]:
# Let's set up name and address lists to hold information about the hotels
name = []
address = []
for i in range(len(austin_hotel["results"])):
    try:
    # Filling the list with data from the API response
        name.append(austin_hotel["results"][i]["name"])
        address.append(austin_hotel["results"][i]["vicinity"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
# Let's create a dataframe to view the list
austin_hotels = pd.DataFrame({"Name": name, "Address": address})
#Display the dataframe
austin_hotels.head()

,Name,Address
0,Austin,Austin
1,The Driskill,"604 Brazos Street, Austin"
2,Hilton Garden Inn Austin Downtown/Convention C...,"500 North Interstate Highway 35, Austin"
3,Sheraton Austin Hotel at the Capitol,"701 East 11th Street, Austin"
4,Hilton Austin,"500 East 4th Street, Austin"


## Using google places API to get nearest hotel for every city in selected dataframe

In [10]:
import pandas as pd
pd.options.mode.chained_assignment = None # Prevents warnings during "chained assignments"
# Create the base url for API search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Create a params dictionary to update each iteration
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
    }
# We shall add latitude and longitude values as we iterate over the dataframe
for index, row in hotel_df.iterrows():
    # get latitude and longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    # create a new "location" key in the parameters dictionary
    params["location"] = f"{lat},{lng}"
    # create a request url and get a .json response
    hotel_address = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index,"Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing hotel name or address... skipping.")
# Location of the output data file
Output_file = "Output_Data/Hotels.csv"        
# Save Data to csv
hotel_df.to_csv(Output_file)
# Display the dataframe
hotel_df.head(15)

,City,Country,Latitude,Longitude,Name
0,Kununurra,AU,-15.77,128.73,Kununurra
1,Benguela,AO,-12.58,13.41,Benguela
2,Weleri,ID,-6.97,110.07,Lebo
3,Ji’an,CN,27.12,114.98,Ji'an
4,Morondava,MG,-20.28,44.28,Morondava
5,Sirte,LY,31.21,16.59,Sirte
6,Kamenka,RU,51.32,42.77,Kamenka
7,Labuhan,ID,-6.88,112.21,Musholla Babut Taubat
8,Erzin,TR,36.96,36.20,Erzin
9,Detchino,RU,54.81,36.31,Detchino


In [11]:
# Using the template add the hotel markers to the heatmap
info_box_template = """ 
<dl>
<dt>Name</dt><dd>{Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store hotel name, city and country from dataframe in the info_box template
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Store the co-ordinates of the hotels in hotel_locations list
hotel_locations = hotel_df[["Latitude", "Longitude"]]
# Defining a marker layer
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
# Adding marker layer to the existing heatmap
fig.add_layer(marker_layer)
# Display figure with marker
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1200px'))

## Using google places API to find natural feature for every city in selected dataframe

In [12]:
# Let's make a copy of the dataframe and keep only the relevant columns
natural_df = df_mycities[["City","Country","Latitude","Longitude"]]
# Creating additional columns for three natural features close to the cities
natural_df.insert(loc = 4,column = "First Feature",value=["" for i in range(natural_df.shape[0])])
natural_df.insert(loc = 5,column = "Second Feature",value=["" for i in range(natural_df.shape[0])])
natural_df.insert(loc = 6,column = "Third Feature",value=["" for i in range(natural_df.shape[0])])

In [13]:
import pandas as pd
pd.options.mode.chained_assignment = None # Prevents warnings during "chained assignments"
# Create the base url for API search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Create a params dictionary to update each iteration
params = {
    "radius": 10000,
    "type": "natural_feature",
    "key": g_key
    }
# We shall add latitude and longitude values as we iterate over the dataframe
print('\033[92m'+'\033[1m'+"------ Starting Data Retrieval ---------"+'\033[0m')
for index, row in hotel_df.iterrows():
    # get latitude and longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    # create a new "location" key in the parameters dictionary
    params["location"] = f"{lat},{lng}"
    # create a request url and get a .json response
    natural_feature = requests.get(base_url, params=params).json()
    try:
            natural_df.loc[index,"First Feature"] = natural_feature["results"][0]["name"]      
    except (KeyError, IndexError):
            print('\033[96m'+"No First feature in index : {} ! Skipping.." .format(index)+'\033[0m')
    try:
            natural_df.loc[index,"Second Feature"] = natural_feature["results"][1]["name"]
    except (KeyError, IndexError):
            print('\033[93m'+"No Second feature in index : {} ! Skipping.." .format(index)+'\033[0m')       
    try:       
            natural_df.loc[index,"Third Feature"] = natural_feature["results"][2]["name"]
    except (KeyError, IndexError):
            print('\033[94m'+"No Third feature in index : {} ! Skipping.." .format(index)+'\033[0m')
print('\033[95m'+'\033[1m'+"------- End of Data Retrieval ------------"+'\033[0m')
# Location of the output data file
Output_file = "Output_Data/Natural_Features.csv"        
# Save Data to csv
natural_df.to_csv(Output_file)
# Display the dataframe
natural_df.head(15)

------ Starting Data Retrieval ---------
No First feature in index : 9 ! Skipping..
No Second feature in index : 9 ! Skipping..
No Third feature in index : 9 ! Skipping..
No Second feature in index : 11 ! Skipping..
No Third feature in index : 11 ! Skipping..
------- End of Data Retrieval ------------


,City,Country,Latitude,Longitude,First Feature,Second Feature,Third Feature
0,Kununurra,AU,-15.77,128.73,Mount Cyril,Mount Cecil,Lily Creek Lagoon
1,Benguela,AO,-12.58,13.41,Praia Morena,Vala do Coringe,Largo do Pioneiro
2,Weleri,ID,-6.97,110.07,Gunung Buntu,Gunung Siwayut,Gunung Santren
3,Ji’an,CN,27.12,114.98,Zhenjun Mountain,Luozishan,Shengangshan
4,Morondava,MG,-20.28,44.28,Morondava Beach,Canel Hellot,Canel Hellot
5,Sirte,LY,31.21,16.59,Gulf of Sirte,Bi'r as Sadiq,Gulf of Sirte
6,Kamenka,RU,51.32,42.77,Dal'niy Kardail,Dal'niy Kardail,Balka Vikhlyayevka
7,Labuhan,ID,-6.88,112.21,Gunung Lembor,Gunung Menjuluk,Gunung Leran
8,Erzin,TR,36.96,36.20,Esek Hill,Uctepeler,Başyurt Hill
9,Detchino,RU,54.81,36.31,,,


In [14]:
# Using the template add the natural features markers to the heatmap
info_box_template = """ 
<dl>
<dt>Name</dt><dd>{First Feature}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store hotel name, city and country from dataframe in the info_box template
natural_info = [info_box_template.format(**row) for index, row in natural_df.iterrows()]
# Store the co-ordinates of the hotels in hotel_locations list
natural_locations = natural_df[["Latitude", "Longitude"]]
# Defining a marker layer
marker_layer = gmaps.marker_layer(natural_locations, info_box_content=natural_info)
# Starting co-ordinates (center of world map)
center_coordinates = (30.0333,31.2333)
fig = gmaps.figure(center=center_coordinates,map_type="HYBRID",zoom_level=2.2,layout=figure_layout)
# Adding marker layer
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1200px'))

## Using google places API to find point-of-interest for every city in selected dataframe

In [15]:
# Let's make a copy of the dataframe and keep only the relevant columns
interest_df = df_mycities[["City","Country","Latitude","Longitude"]]
# Creating additional columns for three natural features close to the cities
interest_df.insert(loc = 4,column = "First Feature",value=["" for i in range(natural_df.shape[0])])
interest_df.insert(loc = 5,column = "Second Feature",value=["" for i in range(natural_df.shape[0])])
interest_df.insert(loc = 6,column = "Third Feature",value=["" for i in range(natural_df.shape[0])])

In [16]:
import pandas as pd
pd.options.mode.chained_assignment = None # Prevents warnings during "chained assignments"
# Create the base url for API search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Create a params dictionary to update each iteration
params = {
    "radius": 10000,
    "type": "point_of_interest",
    "key": g_key
    }
# We shall add latitude and longitude values as we iterate over the dataframe
print('\033[92m'+'\033[1m'+"------ Starting Data Retrieval ---------"+'\033[0m')
for index, row in hotel_df.iterrows():
    # get latitude and longitude from dataframe
    lat = row["Latitude"]
    lng = row["Longitude"]
    # create a new "location" key in the parameters dictionary
    params["location"] = f"{lat},{lng}"
    # create a request url and get a .json response
    interest_feature = requests.get(base_url, params=params).json()
    try:
            interest_df.loc[index,"First Feature"] = interest_feature["results"][0]["name"]      
    except (KeyError, IndexError):
            print('\033[96m'+"No First feature in index : {} ! Skipping.." .format(index)+'\033[0m')
    try:
            interest_df.loc[index,"Second Feature"] = interest_feature["results"][1]["name"]
    except (KeyError, IndexError):
            print('\033[93m'+"No Second feature in index : {} ! Skipping.." .format(index)+'\033[0m')       
    try:       
            interest_df.loc[index,"Third Feature"] = interest_feature["results"][2]["name"]
    except (KeyError, IndexError):
            print('\033[94m'+"No Third feature in index : {} ! Skipping.." .format(index)+'\033[0m')
print('\033[95m'+'\033[1m'+"------- End of Data Retrieval ------------"+'\033[0m')
# Location of the output data file
Output_file = "Output_Data/Point_of_Interest.csv"        
# Save Data to csv
interest_df.to_csv(Output_file)
# Display the dataframe
interest_df.head(15)

------ Starting Data Retrieval ---------
------- End of Data Retrieval ------------


,City,Country,Latitude,Longitude,First Feature,Second Feature,Third Feature
0,Kununurra,AU,-15.77,128.73,Hotel Kununurra,The Kimberley Grande Resort,Discovery Parks - Lake Kununurra
1,Benguela,AO,-12.58,13.41,Hotel Praia Morena,Hotel Luso,Nancy's Guest House
2,Weleri,ID,-6.97,110.07,Pondok Darul Arqom 4 - SMP Muhammadiyah,MTs NU 08 Gemuh,MTs NU 04 Muallimin Weleri
3,Ji’an,CN,27.12,114.98,Gunan Tower,Dizang'an,Lijiacun
4,Morondava,MG,-20.28,44.28,La Case Bambou,Cyber Cool Café,Vezo Hôtel
5,Sirte,LY,31.21,16.59,Sirte Central,Spring Flower Company,Osama Women's Clothes Hall
6,Kamenka,RU,51.32,42.77,Kamenskaya Oosh,Kamenskiy Dom Kul'tury,Rynok S Baychurovo
7,Labuhan,ID,-6.88,112.21,Port Office Brondong,PT Bariscan Global Usaha,Musholla Babut Taubat
8,Erzin,TR,36.96,36.20,Kuzuculu,Artemis Otel,Ertaç Kardeşler Market
9,Detchino,RU,54.81,36.31,Art Hotel Karaskovo,Vorob'yevo,"Ooo ""Eternit Kaluga"""


In [17]:
# Using the template add the natural features markers to the heatmap
info_box_template = """ 
<dl>
<dt>Name</dt><dd>{First Feature}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store hotel name, city and country from dataframe in the info_box template
interest_info = [info_box_template.format(**row) for index, row in interest_df.iterrows()]
# Store the co-ordinates of the hotels in hotel_locations list
interest_locations = interest_df[["Latitude", "Longitude"]]
# Defining a marker layer
marker_layer = gmaps.marker_layer(interest_locations, info_box_content=interest_info)
# Starting co-ordinates (center of world map)
center_coordinates = (30.0333,31.2333)
fig = gmaps.figure(center=center_coordinates,map_type="TERRAIN",zoom_level=2.2,layout=figure_layout)
# Adding marker layer
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1200px'))